In [2]:
import findspark
findspark.init()
from pyspark import SparkContext
sc =SparkContext()

In [4]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf, lit
from pyspark.sql.types import IntegerType
from pyspark import SQLContext
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml import Pipeline
import pandas as pd

spark = SparkSession\
    .builder\
    .appName("Clustering")\
    .getOrCreate()
# dataframe1 = spark.read.json('hdfs://localhost:1234/user/tl2861/hw3/train.json')

sqlContext = SQLContext(sc)
# dataframe1 = sqlContext.read.json('hdfs://localhost:1234/user/tl2861/hw3/train.json')
dataframe1 = sqlContext.read.json('../../../data/AA/wiki_*').withColumn('label',lit(0))
dataframe2 = sqlContext.read.json('../../../data/AAquote/wiki_*').withColumn('label', lit(1))
# dataframe3 = sqlContext.read.json('../../data/AAvoyage/wiki_*').withColumn('label',lit(3))
dataframe = dataframe1.unionAll(dataframe2)
# dataframe1.printSchema()
# dataframe2.printSchema()
dataframe.printSchema()
# dataframe1.show()
#regexTokenizer = RegexTokenizer()
sampledData = dataframe.sampleBy("label", fractions={0: 0.2, 1: 0.2}, seed=0)
# sampledData.show()
# sampledData0 = pd.DataFrame(sampledData)

regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="[^A-Za-z]+", toLowercase=True)
tokenizedData = regexTokenizer.transform(sampledData)

stopWordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
filteredData = stopWordsRemover.transform(tokenizedData)

hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=20)
featurizedData = hashingTF.transform(filteredData)

idf= IDF(inputCol="raw_features", outputCol="features")
idfModel = idf.fit(featurizedData)
data1 = idfModel.transform(featurizedData)

datatext = data1.select('id','features','label')
datatext.show()

root
 |-- id: string (nullable = true)
 |-- text: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- label: integer (nullable = false)

+------+--------------------+-----+
|    id|            features|label|
+------+--------------------+-----+
|205661|(20,[0,1,2,3,4,5,...|    0|
|206148|(20,[0,1,2,3,4,5,...|    0|
|206369|(20,[13],[0.33957...|    0|
|206436|(20,[0,1,2,3,4,5,...|    0|
|207119|(20,[0,1,2,3,4,5,...|    0|
|207153|(20,[0,1,2,3,4,5,...|    0|
|207214|(20,[0,1,2,3,4,5,...|    0|
|207437|(20,[0,1,2,3,4,5,...|    0|
|207482|(20,[0,1,2,3,4,5,...|    0|
|207735|(20,[0,1,2,3,4,5,...|    0|
|208088|(20,[0,1,2,3,4,5,...|    0|
|208519|(20,[0,1,2,3,4,5,...|    0|
|208803|(20,[0,1,2,3,4,5,...|    0|
|208933|(20,[0,1,2,3,4,5,...|    0|
|209059|(20,[0,1,2,3,4,5,...|    0|
|209467|(20,[0,1,2,3,4,5,...|    0|
|209844|(20,[0,1,2,3,4,5,...|    0|
|210549|(20,[0,2,3,4,5,6,...|    0|
|210940|(20,[0,1,2,3,4,5,...|    0|
|210997|(20,[0,1,2,3,

In [ ]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[regexTokenizer, stopWordsRemover, hashingTF,idf])
pipeline_model=pipeline.fit(sampledData)
processed_data = pipeline_model.transform(sampledData)

In [8]:
datatext = datatext.toPandas()


AttributeError: 'DataFrame' object has no attribute 'map'

In [5]:
datatext.toPandas().to_csv("../1/dataCluster.csv", sep = ",", index = False, encoding = "utf-8")

In [6]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans().setK(2).setSeed(4381)
kmModel = kmeans.fit(data1)
wssse = kmModel.computeCost(data1)
print("Within Set Sum of Squared Errors = " + str(wssse))
centers = kmModel.clusterCenters()
print("Cluster Centers: ")
for center in centers:
     print(center)
        
data_kmeans = kmModel.transform(data1).select('id','features','prediction')
data_kmeans.show()
data_kmeans.toPandas().to_csv("../1/dataCluster-kmeans.csv",sep = ",", index = False, encoding = "utf-8")       



Within Set Sum of Squared Errors = 1335214.9809889607
Cluster Centers: 
[24.70595957 21.86310698 28.09276079 24.66477462 31.0944174  18.56860122
 22.48443747 19.4617212  23.95028956 18.58052216 23.5488747  21.43006121
 17.48836579 26.64247329 20.18451575 24.49362609 19.96815644 22.04731022
 19.89367364 21.86714012]
[1.45820232 1.27541215 1.43199953 1.33120029 1.53704863 1.16367569
 1.33369765 1.24050032 1.49248339 1.28095416 1.32525024 1.31310688
 1.16756241 1.47920655 1.32047555 1.39996726 1.30438661 1.2579419
 1.25805494 1.19740135]
+------+--------------------+----------+
|    id|            features|prediction|
+------+--------------------+----------+
|205661|(20,[0,1,2,3,4,5,...|         1|
|206148|(20,[0,1,2,3,4,5,...|         1|
|206369|(20,[13],[0.33957...|         1|
|206436|(20,[0,1,2,3,4,5,...|         1|
|207119|(20,[0,1,2,3,4,5,...|         1|
|207153|(20,[0,1,2,3,4,5,...|         1|
|207214|(20,[0,1,2,3,4,5,...|         1|
|207437|(20,[0,1,2,3,4,5,...|         1|
|207482|